In [1]:
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[tf.nn.conv2d 和 tf.layers.conv2d](https://www.jianshu.com/p/c173976c4792)

tf.nn.conv2d， 需要自行传入初始化好的filter（四个维度），在初始化filter或者说Weights中，已经手动选择了初始化方案，一般用的是tf.truncated_normal。另外tf.nn.conv2d中激活函数需要另外写。

而在tf.layers.conv2d中，只要写入n和size，参数列表中kernel_initializer默认是None， 不清楚这里的参数是如何初始化的？ tf.layers.conv2d中，激活函数是一个参数，不需要另外调用。


tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, name=None)
strides 卷积时在图像每一维的步长，这是一个一维的向量，长度4

In [9]:
# 2-D conv
"""
input [batch, in_height, in_width, in_channels]

filter [filter_height, filter_width, in_channels, out_channels]
filter_shape = [filter_size, embedding_size, 1, num_filters]

output(feature map) [batch, out_height, out_width, out_channels]

一个 epoch（代）是指整个数据集正向反向训练一次。

二维卷积(conv2d)的简化,二维卷积是将一个特征图在width和height两个方向上进行滑动窗操作，对应位置进行相乘求和；
而一维卷积则只是在width或者说height方向上进行滑动窗口并相乘求和。

"""
input = tf.Variable(tf.random_normal([100,60,60,5]))  # one example: [60,60,5]= [width,height,channel]
print(input.shape)
filter = tf.Variable(tf.random_normal([3,3,5,1]))

b = tf.Variable(tf.constant(0.1, shape=[1]), name="b")

"""
    strides: shape=[strides_batch, strides_height, strides_width, strides_channels] ,
    padding：SAME or VALID 是否用0填充，SAME用0填充，VALID不使用0填充
"""
conv = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID')
print(conv.shape)  # (100, 58, 58, 1)

# 一个叫bias的向量加到一个叫value的矩阵上，是向量与矩阵的每一行进行相加，得到的结果和value矩阵大小相同。
# bias的维度必须与value的最后一维相同
h = tf.nn.relu(tf.nn.bias_add(value=conv, bias=b), name="relu")  # 激活函数  W*X + b
print(h.shape)
# Maxpooling over the outputs
pooled = tf.nn.max_pool(
    h,
    ksize=[1, 2, 2, 1],
    strides=[1, 1, 1, 1],
    padding='VALID',
    name="pool")

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(pooled).shape)  # (100, 57, 57, 1)

(100, 60, 60, 5)
(100, 58, 58, 1)
(100, 58, 58, 1)
(100, 57, 57, 1)


In [15]:
# tf.layers.conv2d
print(input.shape)
conv = tf.layers.conv2d(input, filters=2, kernel_size=[3,3],strides=1,padding='VALID')
conv.shape

(100, 60, 60, 5)


TensorShape([Dimension(100), Dimension(58), Dimension(58), Dimension(2)])

In [5]:
# 1-D Conv
input = tf.Variable(tf.random_normal([100,10,2]))  # [10,2] = [seq_len,dim_len]
filter = tf.Variable(tf.random_normal([1,2,1]))

# conv1d = tf.nn.conv1d(input, filter, stride=1, padding='VALID')
conv1d = tf.layers.conv1d(inputs=input,filters=5,kernel_size=3)
print(conv1d.shape)  #(100, 8, 1)  = [batch_size, seq_length-kernel_size+1, num_filter]

# 一个叫bias的向量加到一个叫value的矩阵上，是向量与矩阵的每一行进行相加，得到的结果和value矩阵大小相同。
# bias的维度必须与value的最后一维相同
b = tf.Variable(tf.constant(0.1, shape=[5]), name="b")
h = tf.nn.relu(tf.nn.bias_add(value=conv1d, bias=b), name="relu")  # 激活函数

pooled = tf.reduce_max(h, reduction_indices=[1], name='gmp')
# Maxpooling over the outputs
# pooled = tf.nn.max_pool(
#     h,
#     ksize=[1, 2, 2, 1],
#     strides=[1, 1, 1, 1],
#     padding='VALID',
#     name="pool")

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(pooled).shape)  # 10,4,4,1

Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(100, 8, 5)
(100, 5)
